In [1]:
import pickle
import gensim

In [2]:
#reading the variables from file : dictionary and tdf/corpus
with open(r"C:\Users\kanis\Major-Project\Tokens\tkn01.pkl","rb") as fp:
    data = pickle.load(fp)

dictionary = data[0]
termdocfreq = data[1]
finallemma = data[2]
feedtodict = data[3]

In [3]:
from pprint import pprint

# number of topics
num_topics = 10

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=termdocfreq,
                                       id2word=dictionary,
                                       num_topics=num_topics,random_state=100, chunksize=100, passes=10, per_word_topics=True)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[termdocfreq]

[(0,
  '0.037*"word" + 0.031*"language" + 0.023*"learning" + 0.021*"learn" + '
  '0.015*"help" + 0.014*"thing" + 0.013*"one" + 0.012*"reading" + '
  '0.011*"important" + 0.011*"day"'),
 (1,
  '0.035*"language" + 0.034*"thing" + 0.030*"watch" + 0.030*"show" + '
  '0.029*"movie" + 0.026*"reading" + 0.024*"phrase" + 0.024*"trying" + '
  '0.023*"like" + 0.017*"Possibly"'),
 (2,
  '0.038*"word" + 0.027*"language" + 0.023*"learning" + 0.020*"learn" + '
  '0.017*"help" + 0.013*"important" + 0.012*"thing" + 0.012*"phrase" + '
  '0.011*"reading" + 0.011*"time"'),
 (3,
  '0.033*"language" + 0.027*"learning" + 0.025*"learn" + 0.024*"phrase" + '
  '0.020*"important" + 0.019*"word" + 0.018*"thing" + 0.018*"reading" + '
  '0.015*"understand" + 0.015*"sentence"'),
 (4,
  '0.018*"word" + 0.017*"learning" + 0.016*"language" + 0.013*"learn" + '
  '0.010*"help" + 0.009*"reading" + 0.008*"new" + 0.008*"phrase" + '
  '0.008*"grammar" + 0.008*"time"'),
 (5,
  '0.041*"language" + 0.031*"word" + 0.025*"learni

In [4]:
print("hello world")

hello world


In [5]:
#coherence and score 
from gensim.models import CoherenceModel
# instantiate topic coherence model
import numpy as np
newcorpus = []
for lis in finallemma :
    for w in lis :
        if len(w)>2:
            newcorpus.append(w)

cm = CoherenceModel(model=lda_model, texts=feedtodict, dictionary=dictionary, coherence='c_v')
with np.errstate(invalid='ignore'):
    coherence_lda = cm.get_coherence()
    print('Coherence: ', coherence_lda)                
# getting topic coherence score 


#getting the perplexity
print('\nPerplexity: ', lda_model.log_perplexity(termdocfreq))

Coherence:  0.4157517396267627

Perplexity:  -5.1279096613698245


In [6]:
#import pickle 
#import pyLDAvis
#import os 
#import pyLDAvis.gensim


#num_topics = 10
import pyLDAvis
import pyLDAvis.gensim 

# Visualize the topics
pyLDAvis.enable_notebook()
#vis = pyLDAvis.gensim.prepare(lda_model, termdocfreq, dictionary,sort_topics = 'false')
#vis
#LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))

#LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, termdocfreq, dictionary)
    

In [19]:
import pandas as pd

In [59]:
#Finding the dominant topics 
def format_topics_sentences(ldamodel=lda_model, corpus=termdocfreq, texts=feedtodict):
    # Init output
    
    dp = []
    pc = []
    tk = []
    
    
    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        #print(row_list)
        row = row_list[0] if ldamodel.per_word_topics else row_list 
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                #print(topic_keywords + " "+str(topic_num)+" "+str(prop_topic))
                #sent_topics_df = pd.concat([sent_topics_df, pd.Series([int(topic_num), round(prop_topic,4), topic_keywords])], ignore_index=True)
                #df = pd.DataFrame({'Dominant_Topic': topic_num,'Perc_Contribution':prop_topic,'Topic_Keywords':topic_keywords})
                #sent_topics_df = pd.concat([sent_topics_df, df], ignore_index=True)
                dp.append(topic_num)
                pc.append(prop_topic)
                tk.append(topic_keywords)
            else:
                break
   
    sent_topics_df = pd.DataFrame({'Dominant_Topic' : dp, 'perc_contribution':pc, 'topic_keywords':tk })
    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)
    

df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=termdocfreq, texts=feedtodict)
df_topic_sents_keywords.head(10)
# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(100)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,1,0.549954,"language, thing, watch, show, movie, reading, ...",[Possibly]
1,1,1,0.699966,"language, thing, watch, show, movie, reading, ...","[Possibly, easiest]"
2,2,1,0.774973,"language, thing, watch, show, movie, reading, ...","[Possibly, easiest, thing]"
3,3,1,0.819981,"language, thing, watch, show, movie, reading, ...","[Possibly, easiest, thing, watch]"
4,4,1,0.849985,"language, thing, watch, show, movie, reading, ...","[Possibly, easiest, thing, watch, television]"
...,...,...,...,...,...
95,95,1,0.990998,"language, thing, watch, show, movie, reading, ...","[Possibly, easiest, thing, watch, television, ..."
96,96,1,0.991087,"language, thing, watch, show, movie, reading, ...","[Possibly, easiest, thing, watch, television, ..."
97,97,1,0.991175,"language, thing, watch, show, movie, reading, ...","[Possibly, easiest, thing, watch, television, ..."
98,98,1,0.991260,"language, thing, watch, show, movie, reading, ...","[Possibly, easiest, thing, watch, television, ..."
